In [89]:
import pandas as pd
import requests

In [44]:
# TODO OMIT THIS
import glob
import os

mep_data_folder_path_abs = r'C:\Users\dennis\Desktop\CHES_DATA\MEP_DATA_EU_OPENDATA\downloads_eu_opendata_portal'
# Find all CSV files in the folder, read and combine them
csv_files = glob.glob(os.path.join(mep_data_folder_path_abs, "*.csv"))
df_list = [pd.read_csv(file) for file in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)
print(combined_df.info())
# Save merged CSV
combined_df.to_csv(r"C:\Users\dennis\Desktop\CHES_DATA\MEP_DATA_EU_OPENDATA\merged_mep_meta_data.csv", index=False)
print(f"Merged {len(csv_files)} files into merged_mep_meta_data.csv")

####
####

# OMIT: investigate dataframes, check if all columns are the same
last_list = list(df_list[1].columns)
print("colum names:", last_list)
print(df_list[1].head())
print(df_list[-1].iloc[1])
for df in df_list:
    columns = list(df.columns)
    if columns != last_list:
        print("ERROR")
    last_list = columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8457 entries, 0 to 8456
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   mep_identifier                 8457 non-null   int64 
 1   mep_honorific_prefix           8455 non-null   object
 2   mep_given_name                 8450 non-null   object
 3   mep_family_name                8457 non-null   object
 4   mep_official_given_name        448 non-null    object
 5   mep_official_family_name       448 non-null    object
 6   mep_gender                     8457 non-null   object
 7   mep_citizenship                8457 non-null   object
 8   mep_place_of_birth             8225 non-null   object
 9   mep_birthday                   8158 non-null   object
 10  mep_death_date                 1187 non-null   object
 11  mep_image                      8457 non-null   object
 12  mep_email                      1312 non-null   object
 13  mep

**Create MEP and National Party Datasets**

First, fetch data of MEPs for all legislative terms

In [108]:
mep_data = {}
# we have 11 terms in total (but we start with the 5th term, i.e. 1999-2004)
for term in range(5,11):
    response = requests.get("https://data.europarl.europa.eu/api/v2/meps",
                            params={"parliamentary-term": term, "format": "application/ld+json"})
    if response.status_code == 200:
        mep_data[term] = response.json()['data']
    else:
        print(f"Couldn't find data on LT: {term}")
print("Finished without error.")
# merge everything into one dataframe
mep_df = pd.DataFrame([{**mep, "period": term} for term, data in mep_data.items() for mep in data])


Finished


In [110]:
# perform some sanity checks:
# how many (unique) MEPs do we have in total? (should not exceed total amount of seats held ~ 4,341)
print("Amount of entries in mep_df:",mep_df.shape[0])
amount_unique_mep_total = len(set(mep_df['id'].to_list()))
print(f"We have in total {amount_unique_mep_total} unique MEPs over terms 5-10.")
print(mep_df.head())

Amount of entries in mep_df: 5186
We have in total 3294 unique MEPs over terms 5-10.
             id    type identifier                          label  \
0   person/2278  Person       2278                     Paul RÜBIG   
1   person/1344  Person       1344  Carles-Alfred GASÒLIBA I BÖHM   
2   person/1443  Person       1443         James (Jim) FITZSIMONS   
3   person/4436  Person       4436                Gianni PITTELLA   
4  person/23202  Person      23202             Peder WACHTMEISTER   

        familyName      givenName        sortLabel  period officialFamilyName  \
0            Rübig           Paul            RUBIG       5                NaN   
1  Gasòliba i Böhm  Carles-Alfred  GASOLIBA I BÖHM       5                NaN   
2       Fitzsimons    James (Jim)       FITZSIMONS       5                NaN   
3         Pittella         Gianni         PITTELLA       5                NaN   
4     Wachtmeister          Peder     WACHTMEISTER       5                NaN   

  officialGiv

Now use the MEP's identifier to fetch the relevant meta data

In [119]:
# get the meta data on the MEP's
metadata = []
identifiers = list(set(mep_df['identifier']))
batch_size = 128
idx = 0
while idx < len(identifiers):
    print(f"idx: {idx}")
    batch = identifiers[idx: idx + batch_size]
    response = requests.get(f"https://data.europarl.europa.eu/api/v2/meps/{','.join(batch)}",
                                params={"format": "application/ld+json"})

    if response.status_code == 200:
        metadata.extend(response.json()['data'])
    else:
        print(f"An error occurred: {response.status_code}")

    idx += batch_size
print("Finished without error.")
# create dataframe
mep_metadata = pd.DataFrame(metadata)

idx: 0
idx: 128
idx: 256
idx: 384
idx: 512
idx: 640
idx: 768
idx: 896
idx: 1024
idx: 1152
idx: 1280
idx: 1408
idx: 1536
idx: 1664
idx: 1792
idx: 1920
idx: 2048
idx: 2176
idx: 2304
idx: 2432
idx: 2560
idx: 2688
idx: 2816
idx: 2944
idx: 3072
idx: 3200
Finished without error.


In [120]:
# perform some sanity checks
#assert len(metadata) == amount_unique_mep_total
print(f"We have metadata on {len(metadata)} MEPs")
mep_metadata.head()

We have metadata on 3294 MEPs


,id,type,identifier,label,notation_codictPersonId,bday,hasGender,hasHonorificPrefix,hasMembership,citizenship,...,upperFamilyName,upperGivenName,hasEmail,deathDate,officialFamilyName,officialGivenName,upperOfficialFamilyName,upperOfficialGivenName,account,homepage
0,person/1038,Person,1038,Karl-Heinz FLORENZ,1038,1947-10-22,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/1038-f-144759', 'type': 'M...",http://publications.europa.eu/resource/authori...,...,FLORENZ,KARL-HEINZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,person/1062,Person,1062,Karin JUNKER,1062,1940-12-24,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/1062-f-70176', 'type': 'Me...",http://publications.europa.eu/resource/authori...,...,JUNKER,KARIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,person/107973,Person,107973,Tarja CRONBERG,107973,1943-06-29,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/107973-f-135664', 'type': ...",http://publications.europa.eu/resource/authori...,...,CRONBERG,TARJA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,person/112755,Person,112755,Andrej PLENKOVIĆ,112755,1970-04-08,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/112755-f-139724', 'type': ...",http://publications.europa.eu/resource/authori...,...,PLENKOVIĆ,ANDREJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,person/1175,Person,1175,Gérard CAUDRON,1175,1945-02-27,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/1175-f-104118', 'type': 'M...",http://publications.europa.eu/resource/authori...,...,CAUDRON,GÉRARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now clean up the dataframe

In [121]:
# replace URI entries with actual data for gender, citizenship and prefix
mep_metadata['gender'] = mep_metadata['hasGender'].apply(lambda gstr: gstr[len("http://publications.europa.eu/resource/authority/human-sex/"):])

mep_metadata['citizenship'] = mep_metadata['citizenship'].apply(lambda gstr: gstr[len("http://publications.europa.eu/resource/authority/country/"):])

mep_metadata['honorific_prefix'] =  mep_metadata['hasHonorificPrefix'].apply(lambda prefix: prefix[len("http://publications.europa.eu/resource/authority/honorific/"):] if isinstance(prefix, str) else prefix)

# get rid of unnecessary columns
#mep_metadata = mep_metadata.drop('hasGender', axis='columns')
#mep_metadata = mep_metadata.drop('hasHonorificPrefix', axis='columns')

mep_metadata.head()

,id,type,identifier,label,notation_codictPersonId,bday,hasGender,hasHonorificPrefix,hasMembership,citizenship,...,hasEmail,deathDate,officialFamilyName,officialGivenName,upperOfficialFamilyName,upperOfficialGivenName,account,homepage,gender,honorific_prefix
0,person/1038,Person,1038,Karl-Heinz FLORENZ,1038,1947-10-22,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/1038-f-144759', 'type': 'M...",DEU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MALE,MR
1,person/1062,Person,1062,Karin JUNKER,1062,1940-12-24,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/1062-f-70176', 'type': 'Me...",DEU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,MS
2,person/107973,Person,107973,Tarja CRONBERG,107973,1943-06-29,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/107973-f-135664', 'type': ...",FIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FEMALE,MS
3,person/112755,Person,112755,Andrej PLENKOVIĆ,112755,1970-04-08,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/112755-f-139724', 'type': ...",HRV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MALE,MR
4,person/1175,Person,1175,Gérard CAUDRON,1175,1945-02-27,http://publications.europa.eu/resource/authori...,http://publications.europa.eu/resource/authori...,"[{'id': 'membership/1175-f-104118', 'type': 'M...",FRA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MALE,MR


Now lets take care of membership and party affiliations (i.e. the "hasMembership" column)

In [127]:
def extract_national_political_group(memberships):
    national_parties = [{"id": membership.get('organization', '')[len("org/"):],
                         "startDate": membership['memberDuring'].get('startDate'),
                         "endDate": membership['memberDuring'].get('endDate'),
                         "role": membership.get('role')
                         } for membership in memberships if
                         membership.get('membershipClassification', "") == 'def/ep-entities/NATIONAL_POLITICAL_GROUP']
    return national_parties

def extract_eu_political_group(memberships):
    european_parties = [{"id": membership.get('organization', '')[len("org/"):],
                         "startDate": membership['memberDuring'].get('startDate'),
                         "endDate": membership['memberDuring'].get('endDate'),
                         "role": membership.get('role')
                         } for membership in memberships if
                         membership.get('membershipClassification', "") == 'def/ep-entities/EU_POLITICAL_GROUP']
    return european_parties

In [132]:
mep_metadata['national_parties'] = mep_metadata['hasMembership'].apply(extract_national_political_group)
mep_metadata['eu_parties'] = mep_metadata['hasMembership'].apply(extract_eu_political_group)
print(mep_metadata.head())
print(mep_metadata.describe())

              id    type identifier               label  \
0    person/1038  Person       1038  Karl-Heinz FLORENZ   
1    person/1062  Person       1062        Karin JUNKER   
2  person/107973  Person     107973      Tarja CRONBERG   
3  person/112755  Person     112755    Andrej PLENKOVIĆ   
4    person/1175  Person       1175      Gérard CAUDRON   

  notation_codictPersonId        bday  \
0                    1038  1947-10-22   
1                    1062  1940-12-24   
2                  107973  1943-06-29   
3                  112755  1970-04-08   
4                    1175  1945-02-27   

                                           hasGender  \
0  http://publications.europa.eu/resource/authori...   
1  http://publications.europa.eu/resource/authori...   
2  http://publications.europa.eu/resource/authori...   
3  http://publications.europa.eu/resource/authori...   
4  http://publications.europa.eu/resource/authori...   

                                  hasHonorificPrefix  \
0  ht

Now fetch the relevant meta data on all occurring national parties

In [133]:
# get all occurring party id's
national_party_ids = set(mep_metadata['national_parties'].explode().apply(lambda org: org['id']))
len(national_party_ids)

# fetch the data on national parties
national_parties = []
national_party_ids = list(national_party_ids)
batch_size = 128
idx = 0
while idx < len(national_party_ids):
    print(f"idx: {idx}")
    batch = national_party_ids[idx: idx + batch_size]
    response = requests.get(f"https://data.europarl.europa.eu/api/v2/corporate-bodies/{','.join(batch)}",
                                params={"format": "application/ld+json"})

    if response.status_code == 200:
        national_parties.extend(response.json()['data'])
    else:
        print(f"An error occured: {response.status_code}")

    idx += batch_size
print("Finished without error.")

idx: 0
idx: 128
idx: 256
idx: 384
idx: 512
idx: 640
idx: 768
idx: 896
idx: 1024
idx: 1152
idx: 1280
idx: 1408
idx: 1536
Finished without error.


In [134]:
national_parties_df = pd.DataFrame(national_parties)
national_parties_df.head()

,id,type,represents,identifier,source,temporal,label,altLabel,notation_providerTemporalBodyId,notation_codictBodyId,prefLabel,classification,isVersionOf
0,org/1030,Organization,[http://publications.europa.eu/resource/author...,1030,EU_PARLIAMENT,"{'id': 'time-period/19990720-20040719', 'type'...",parteilos,"{'pt': 'parteilos', 'nl': 'parteilos', 'sv': '...",1030,1030,"{'en': 'parteilos', 'fi': 'parteilos', 'nl': '...",def/ep-entities/NATIONAL_POLITICAL_GROUP,NaN
1,org/1057,Organization,[http://publications.europa.eu/resource/author...,1057,EU_PARLIAMENT,"{'id': 'time-period/19850101-19921231', 'type'...",PVV,"{'pl': 'PVV', 'ga': 'PVV', 'sl': 'PVV', 'sk': ...",1057,1057,"{'ro': 'Partij voor vrijheid en vooruitgang', ...",def/ep-entities/NATIONAL_POLITICAL_GROUP,NaN
2,org/1062,Organization,[http://publications.europa.eu/resource/author...,1062,EU_PARLIAMENT,"{'id': 'time-period/20011001-20040719', 'type'...",CD&V,"{'et': 'CD&V', 'es': 'CD&V', 'bg': 'CD&V', 'el...",1062,1062,"{'en': 'Christen-Democratisch & Vlaams', 'hr':...",def/ep-entities/NATIONAL_POLITICAL_GROUP,NaN
3,org/1145,Organization,[http://publications.europa.eu/resource/author...,1145,EU_PARLIAMENT,"{'id': 'time-period/20030417-20040719', 'type'...",UP,"{'nl': 'UP', 'ro': 'UP', 'da': 'UP', 'pl': 'UP...",1145,1145,"{'hu': 'Unia Pracy', 'fr': 'Unia Pracy', 'it':...",def/ep-entities/NATIONAL_POLITICAL_GROUP,NaN
4,org/1146,Organization,[http://publications.europa.eu/resource/author...,1146,EU_PARLIAMENT,"{'id': 'time-period/20030417-20040719', 'type'...",PSL,"{'sv': 'PSL', 'fi': 'PSL', 'ga': 'PSL', 'sl': ...",1146,1146,"{'lv': 'Polskie Stronnictwo Ludowe', 'hr': 'Po...",def/ep-entities/NATIONAL_POLITICAL_GROUP,NaN


Do the same with european parties

In [135]:
# get all occurring eu party id's
eu_party_ids = set(mep_metadata['eu_parties'].explode().apply(lambda org: org['id']))
len(eu_party_ids)

# fetch the data on european parties
eu_parties = []
eu_party_ids = list(eu_party_ids)
batch_size = 128
idx = 0
while idx < len(eu_party_ids):
    print(f"idx: {idx}")
    batch = eu_party_ids[idx: idx + batch_size]
    response = requests.get(f"https://data.europarl.europa.eu/api/v2/corporate-bodies/{','.join(batch)}",
                                params={"format": "application/ld+json"})

    if response.status_code == 200:
        eu_parties.extend(response.json()['data'])
    else:
        print(f"An error occured: {response.status_code}")

    idx += batch_size
print("Finished without error.")

idx: 0
Finished without error.


In [136]:
eu_parties_df = pd.DataFrame(eu_parties)
eu_parties_df.head()

,id,type,identifier,isVersionOf,source,temporal,label,altLabel,notation_providerTemporalBodyId,notation_codictBodyId,prefLabel,classification
0,org/1050,Organization,1050,org/CD,EU_PARLIAMENT,"{'id': 'time-period/19780314-19790716', 'type'...",CD,{'sl': 'Skupina krščanskih demokratov (skupina...,1050,1050,{'ga': 'Grúpa na nDaonlathaithe Críostaí (Grúp...,def/ep-entities/EU_POLITICAL_GROUP
1,org/1229,Organization,1229,org/L,EU_PARLIAMENT,"{'id': 'time-period/19790717-19851212', 'type'...",L,"{'es': 'Democrático y Liberal', 'nl': 'Liberal...",1229,1229,"{'pl': 'Grupa Liberalna i Demokratyczna', 'lt'...",def/ep-entities/EU_POLITICAL_GROUP
2,org/1533,Organization,1533,org/PPE-DE,EU_PARLIAMENT,"{'id': 'time-period/20040720-20090713', 'type'...",PPE-DE,{'sv': 'Europeiska folkpartiet (kristdemokrate...,1533,1533,{'lv': 'Eiropas Tautas Partijas (Kristīgo demo...,def/ep-entities/EU_POLITICAL_GROUP
3,org/1534,Organization,1534,org/PSE,EU_PARLIAMENT,"{'id': 'time-period/20040720-20090713', 'type'...",PSE,"{'bg': 'Социалисти', 'de': 'Europäische Sozial...",1534,1534,{'ro': 'Grupul Socialist din Parlamentul Europ...,def/ep-entities/EU_POLITICAL_GROUP
4,org/1537,Organization,1537,org/GUE-NGL,EU_PARLIAMENT,"{'id': 'time-period/20040720-20090713', 'type'...",GUE/NGL,{'sk': 'Európska zjednotená ľavica - Nordická ...,1537,1537,{'lt': 'Europos vieningųjų kairiųjų jungtinė f...,def/ep-entities/EU_POLITICAL_GROUP
